In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader    
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.utils as vutils

import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
import glob

In [3]:
from os import listdir


data_dir = "/home/bhavik/neonLabs-ai/personal/farmbot/deep_learning/dataset"

train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

train_imgs = os.listdir(train_dir + '/images')
test_imgs = os.listdir(test_dir + '/images')
valid_imgs = os.listdir(valid_dir + '/images')

train_data = []
test_data = []
valid_data = []

train_labels = []
test_labels = []
valid_labels = []

print(len(os.listdir(train_dir + '/images')))
print(len(os.listdir(test_dir + '/images')))
print(len(os.listdir(valid_dir + '/images')))

2469
118
235


In [ ]:
train_data = glob.glob('/home/bhavik/neonLabs-ai/personal/farmbot/deep_learning/agri_data/data/*.jpeg')
train_labels = [i.rsplit('.',maxsplit=1)[0]+'.txt' for i in train_data]

In [6]:
## load dataset and transform into tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu
device

device(type='cpu')

In [13]:
class WeedDataset(Dataset):
    
    def __init__(self, imgs, labels, mode= "train", transforms=None):
        super().__init__()
        self.imgs = imgs
        self.labels = labels
        self.mode = mode
        self.transforms = transforms

    def __getitem__(self, index):
        img_name = self.imgs[index]
        label_boxs = self.labels[index]
        img = Image.open(train_dir + img_name ).resize((224,224,3))

        if self.mode == "train" or self.mode == "val":
            
            bboxs = []
            i = 0
            
            for i in range(len(label_boxs)):
                for j in range(len(label_boxs[i])//5):
                    bboxs.append(label_boxs[i][j])
            
            for l in label_boxs:
                bboxs.append(l[i:i+4])
                i +=4

            bboxs = torch.tensor(bboxs, dtype=torch.float32)
            
            img = self.transforms(img)

            return img, bboxs

        elif self.mode == "test":
            img = self.transforms(img)

    def __getlen__(self):
        return len(self.imgs)

In [14]:
## Load labels 

train_labels = load_labels(train_dir)
test_labels = load_labels(test_dir)
valid_labels = load_labels(valid_dir)

In [16]:
## Load and transform dataset into tensors

train_dataset = WeedDataset(train_imgs, train_labels)
test_dataset = WeedDataset(test_imgs, test_labels, mode="test")

In [ ]:
# Load dataset using dataLoaders

train_data_loader = DataLoader(
    dataset = train_dataset,
    shuffle=True
)

In [ ]:
for imgs, labels in train_data_loader:
    plt.imshow(imgs[1])
    print(labels[1])
    break    

In [ ]:
## TODO : Research on neural network architecture using torch for embedded device, 
# optimised for detection speed and accuracy

# 

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# create training pipeline using net class
net = Net()

In [10]:
# create loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# train the network
for epoch in range(1):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(test_data, 0):

        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        
torch.save(net.state_dict(), "/home/bhavik/neonLabs-ai/personal/farmbot/deep_learning/saved_models")
print('Finished Training')

In [ ]:
# test the network on the test data

## Documentation

#### Architecture of CNN for bi-classification

**Test performance**
 
* AlexNet
* GoogleNet
* 